In [85]:
import json
import vk_api
import os
import shutil
from pathlib import Path
import time
import datetime
from urllib.parse import urlparse
from vk_api.vk_api import VkApiMethod
from typing import TextIO, Optional
from tqdm import tqdm
from dotenv import load_dotenv


In [86]:
BASE = Path().cwd()
RESOURCES_DIR = BASE.joinpath("resources")
SCRAPPED_DATA_DIR = BASE.joinpath("scrapped_data")


Задаём конфигурацию

In [87]:
load_dotenv()

# Сервисный ключ доступа
VK_SERVICE_TOKEN = os.getenv("VK_SERVICE_TOKEN")
if VK_SERVICE_TOKEN is None:
    raise ValueError("❌ В .env файле не задан VK_SERVICE_TOKEN")

# Имя входного файла
INPUT_FILE = RESOURCES_DIR.joinpath("vk_urls.json")

# Файл для сохранения результатов
OUTPUT = SCRAPPED_DATA_DIR.joinpath("vk_scrapped.jsonl")

# Дата, НАЧИНАЯ С КОТОРОЙ скраппить посты (Unix timestamp)
# Скрипт собирает все посты от текущего момента назад до этой даты.
# None = скраппить все посты за всё время
# Пример: 1609459200 для 2021-01-01
# Можно использовать: int(datetime.datetime(2020, 1, 1).timestamp())
CUTOFF_DATE = None


Дополнительные переменные

In [88]:
# Количество постов для скачивания (максимум 100 за один запрос)
POSTS_PER_REQUEST = 100


Helper-функции

In [89]:
def get_group_domain(url):
    """Извлекает domain группы из ссылки"""
    return urlparse(url).path.strip("/")


In [90]:
def get_posts(vk: VkApiMethod, domain: str, count: int, offset: int):
    # Добавляем filter='owner' чтобы получать посты именно от имени группы,
    # а не все подряд (хотя по умолчанию usually 'all').
    response = vk.wall.get(domain=domain, count=count, offset=offset, filter="owner")
    posts = response.get("items", [])
    return posts


def to_output_dict(post: dict, name: str, link: str) -> dict:
    result = dict()
    result["url"] = link
    result["name"] = name
    result["content"] = post.get("text", " ")

    result["date"] = post.get("date")
    result["collection_date"] = int(time.time())

    # Дополнительные поля для отладки, если нужно
    if "is_pinned" in post:
        result["is_pinned"] = post["is_pinned"]

    return result


def collect_data(
    vk: VkApiMethod,
    domain: str,
    title: str,
    link: str,
    out: TextIO,
    batch_size: int = 100,
    cutoff_date: Optional[int] = None,
) -> tuple[Optional[int], Optional[int]]:
    """
    Собирает данные и возвращает (min_date, max_date) для собранных постов
    """
    offset = 0
    saved_count = 0
    should_stop = False
    min_date = None
    max_date = None

    # Итеративно скачиваем посты, пока они есть и не достигли cutoff_date
    with tqdm(desc=f"Fetching {title}") as pbar:
        while True:
            if should_stop:
                break

            posts = get_posts(vk, domain, count=batch_size, offset=offset)

            # Если постов нет, значит дошли до конца
            if not posts:
                break

            for post in posts:
                post_date = post.get("date")
                is_pinned = post.get("is_pinned", 0) == 1

                # Важно: закрепленный пост (pinned) может быть старым.
                # Мы его сохраняем, но НЕ прерываем сбор, если он старше cutoff_date.
                # Прерываем только на ОБЫЧНЫХ постах (хронологических).
                if not is_pinned:
                    # Проверяем, не старше ли пост, чем cutoff_date
                    if cutoff_date is not None and post_date < cutoff_date:
                        should_stop = True
                        break  # Break inner loop

                # Обновляем даты (включая pinned в статистике)
                if min_date is None or post_date < min_date:
                    min_date = post_date
                if max_date is None or post_date > max_date:
                    max_date = post_date

                # Сохраняем
                out_post = to_output_dict(post, title, link)
                json_line = json.dumps(out_post, ensure_ascii=False)

                out.write(json_line + "\n")
                saved_count += 1

            # Принудительно сбрасываем буфер на диск
            out.flush()

            offset += len(posts)
            pbar.update(len(posts))

            # Небольшая пауза
            time.sleep(0.3)

    # Форматирование дат для красивого вывода в лог
    min_str = (
        datetime.datetime.fromtimestamp(min_date).strftime("%Y-%m-%d %H:%M:%S")
        if min_date is not None
        else "N/A"
    )
    max_str = (
        datetime.datetime.fromtimestamp(max_date).strftime("%Y-%m-%d %H:%M:%S")
        if max_date is not None
        else "N/A"
    )

    print(f"✅ Сохранено {saved_count} постов. Диапазон дат: {min_str} - {max_str}")

    return min_date, max_date


In [91]:
def main():
    # 1. Авторизация
    try:
        vk_session = vk_api.VkApi(token=VK_SERVICE_TOKEN)
        vk: VkApiMethod = vk_session.get_api()
    except Exception as e:
        print(f"❌ Ошибка авторизации: {e}")
        return

    # 2. Чтение списка групп
    try:
        with open(INPUT_FILE, "r", encoding="utf-8") as f:
            groups_dict = json.load(f)
    except FileNotFoundError:
        print(f"❌ Файл {INPUT_FILE} не найден.")
        return

    cutoff_info = ""
    if CUTOFF_DATE is not None:
        cutoff_info = f" (с {datetime.datetime.fromtimestamp(CUTOFF_DATE)})"

    print(
        f"Найдено групп: {len(groups_dict)}. Начинаем сбор в {OUTPUT}{cutoff_info}..."
    )

    # Отслеживаем общие минимальную и максимальную дату
    global_min_date = None
    global_max_date = None

    # Используем 'w' для перезаписи файла при новом запуске.
    # Файл vk_scrapped.jsonl будет содержать актуальные результаты прогона.
    with open(OUTPUT, "w", encoding="utf-8") as f_out:
        for title, link in groups_dict.items():
            domain = get_group_domain(link)
            print(f"Downloading: {title}...", end=" ")

            try:
                min_date, max_date = collect_data(
                    vk,
                    domain,
                    title,
                    link,
                    f_out,
                    POSTS_PER_REQUEST,
                    CUTOFF_DATE,
                )

                # Обновляем глобальные даты
                if min_date is not None:
                    if global_min_date is None or min_date < global_min_date:
                        global_min_date = min_date
                if max_date is not None:
                    if global_max_date is None or max_date > global_max_date:
                        global_max_date = max_date

            except vk_api.exceptions.ApiError as e:
                print(f"\n⚠️ Ошибка API ({title}): {e}")
            except Exception as e:
                print(f"\n⚠️ Ошибка ({title}): {e}")

    # Создаём КОПИЮ файла с диапазоном дат в названии, оставляя оригинал на месте
    if global_min_date is not None and global_max_date is not None:
        min_date_str = datetime.datetime.fromtimestamp(global_min_date).strftime(
            "%Y-%m-%d"
        )
        max_date_str = datetime.datetime.fromtimestamp(global_max_date).strftime(
            "%Y-%m-%d"
        )

        # Формируем имя с датами
        new_name = OUTPUT.stem + f"_{min_date_str}_to_{max_date_str}" + OUTPUT.suffix
        new_path = OUTPUT.parent / new_name

        # Копируем файл вместо переименования
        shutil.copy2(OUTPUT, new_path)

        print("\n🎉 Готово! Данные сохранены в:")
        print(f"   1. {OUTPUT} (основной файл)")
        print(f"   2. {new_path} (архивная копия с датами)")
        print(f"   Диапазон: с {min_date_str} по {max_date_str}")
    else:
        print(f"\n🎉 Готово! Все данные сохранены в {OUTPUT}")


if __name__ == "__main__":
    main()


Найдено групп: 1. Начинаем сбор в /Users/white/shad/Scrapper/scrapped_data/vk_scrapped.jsonl...
Downloading: Новосибирский государственный университет | НГУ... 

Fetching Новосибирский государственный университет | НГУ: 10797it [04:46, 37.69it/s]

✅ Сохранено 10797 постов. Диапазон дат: 2012-04-03 12:48:48 - 2026-01-19 13:11:00

🎉 Готово! Данные сохранены в:
   1. /Users/white/shad/Scrapper/scrapped_data/vk_scrapped.jsonl (основной файл)
   2. /Users/white/shad/Scrapper/scrapped_data/vk_scrapped_2012-04-03_to_2026-01-19.jsonl (архивная копия с датами)
   Диапазон: с 2012-04-03 по 2026-01-19
